### parsing functions
using - https://github.com/tree-sitter-grammars/tree-sitter-glsl
basic usage: https://github.com/tree-sitter/py-tree-sitter

In [100]:
import tree_sitter_glsl as tsglsl
import tree_sitter
from tree_sitter import Language, Parser

GLSL_LANGUAGE = Language(tsglsl.language())

parser = Parser(GLSL_LANGUAGE)

In [101]:
example_code = """
// comment directly infront of a function
float red(float intensity){
    // comment inside the function body, but the top
    itensity = max(itensity, 1.0)

    // comment deeper inside the function body
    return vec3(1.0,0.0,0.0) * intensity;
}

// comment one before
// comment two before
float blue(float intensity){
    itensity = max(itensity, 1.0)

    return vec3(0.0,0.0,1.0) * intensity;
}


void mainImage( out vec4 fragColor, in vec2 fragCoord )
{
    vec2 uv = fragCoord/iResolution.xy;

    vec3 col = mix(red(0.5), blue(0.3), 0.8);
    fragColor = vec4(col,1.0);
}
"""

tree = parser.parse(bytes(example_code, encoding="utf-8"))
tree

In [102]:
root_node = tree.root_node
for child in root_node.children:
    if child.type == "function_definition":
        print(f"{child.text.decode(encoding='utf-8')}\n##byte range is {child.start_byte}:{child.end_byte}")
        print(str(child))

float red(float intensity){
    // comment inside the function body, but the top
    itensity = max(itensity, 1.0)

    // comment deeper inside the function body
    return vec3(1.0,0.0,0.0) * intensity;
}
##byte range is 43:249
(function_definition type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list (parameter_declaration type: (primitive_type) declarator: (identifier)))) body: (compound_statement (comment) (expression_statement (assignment_expression left: (identifier) right: (call_expression function: (identifier) arguments: (argument_list (identifier) (number_literal)))) (comment) (MISSING ";")) (return_statement (binary_expression left: (call_expression function: (identifier) arguments: (argument_list (number_literal) (number_literal) (number_literal))) right: (identifier)))))
float blue(float intensity){
    itensity = max(itensity, 1.0)

    return vec3(0.0,0.0,1.0) * intensity;
}
##byte range is 295:402
(function_definit

In [103]:
# https://tree-sitter.github.io/tree-sitter/using-parsers#query-syntax

query = GLSL_LANGUAGE.query(
        """
        (
        (comment)* @comments.before
        (function_definition 
            type: (primitive_type) @function.type
            declarator: (function_declarator) @function.declarator
            body: (compound_statement
                (comment)* @comments.body
                ) @function.body
        )
        )
        """
    )

matches = query.matches(tree.root_node)
for match in matches:
    print(match[1]["comments.before"])
    print(match[1]["function.type"].text)
    print(match[1]["function.declarator"].text)
    print(match[1]["function.body"].text)
    print(match[1]["comments.body"])
    print()

[<Node type=comment, start_point=(1, 0), end_point=(1, 41)>]
b'float'
b'red(float intensity)'
b'{\n    // comment inside the function body, but the top\n    itensity = max(itensity, 1.0)\n\n    // comment deeper inside the function body\n    return vec3(1.0,0.0,0.0) * intensity;\n}'
[<Node type=comment, start_point=(3, 4), end_point=(3, 52)>]

[<Node type=comment, start_point=(1, 0), end_point=(1, 41)>]
b'float'
b'blue(float intensity)'
b'{\n    itensity = max(itensity, 1.0)\n\n    return vec3(0.0,0.0,1.0) * intensity;\n}'


KeyError: 'comments.top_body'

In [ ]:
from pprint import pprint

pprint(str(root_node))

('(translation_unit (comment) (function_definition type: (primitive_type) '
 'declarator: (function_declarator declarator: (identifier) parameters: '
 '(parameter_list (parameter_declaration type: (primitive_type) declarator: '
 '(identifier)))) body: (compound_statement (comment) (expression_statement '
 '(assignment_expression left: (identifier) right: (call_expression function: '
 '(identifier) arguments: (argument_list (identifier) (number_literal)))) '
 '(comment) (MISSING ";")) (return_statement (binary_expression left: '
 '(call_expression function: (identifier) arguments: (argument_list '
 '(number_literal) (number_literal) (number_literal))) right: (identifier))))) '
 '(function_definition type: (primitive_type) declarator: (function_declarator '
 'declarator: (identifier) parameters: (parameter_list (parameter_declaration '
 'type: (primitive_type) declarator: (identifier)))) body: (compound_statement '
 '(expression_statement (assignment_expression left: (identifier) right: 